<a href="https://colab.research.google.com/github/gmarchiello/Giuseppe/blob/main/Retrieve_data_from_API_Analyzing_Music_Trends_Across_Decades_Insights_from_Spotify's_%22All_Out%22_Playlists.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Retrieving Data - Analyzing Music Trends Across Decades: Insights from Spotify's "All Out" Playlists

The "All Out" series of playlists on Spotify, such as "All Out 80s," "All Out 90s," etc., are carefully curated collections that highlight popular and iconic tracks from specific decades.

To retrieve data for cleaning and analysis, we will use the Spotify API. We will then save the retrieved data in a JSON file for further processing.


## Install Spotipy


In [ ]:
# Spotipy is a Python library for accessing the Spotify Web API
!pip install spotipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.6/255.6 kB 11.7 MB/s eta 0:00:00


## Import packages

In [ ]:
# Import the json module for working with JSON data
import json

# Import the spotipy library to interact with the Spotify Web API
import spotipy

import time

# Import SpotifyClientCredentials from spotipy.oauth2 for authentication
# This class is used to handle authentication with Spotify's API using client credentials
from spotipy.oauth2 import SpotifyClientCredentials

## Insert API credentials

In [ ]:
# Insert your Spotify API credentials
CLIENT_ID = input('Insert your Spotify Client ID: ')
CLIENT_SECRET = input('Insert your Spotify Client Secter: ')

# Authenticate with Spotify API
client_credentials_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

## Select spotify "All of" playlist URIs
The "All of ..." spotify playlist concentrate the most populare songs of a decade.

In [ ]:
#URIs of 'All Out' Playlists for each decade
playlists = {
    '1950s': 'spotify:playlist:37i9dQZF1DWSV3Tk4GO2fq',
    '1960s': 'spotify:playlist:37i9dQZF1DXaKIA8E7WcJj',
    '1970s': 'spotify:playlist:37i9dQZF1DWTJ7xPn4vNaz',
    '1980s': 'spotify:playlist:37i9dQZF1DX4UtSsGT1Sbe',
    '1990s': 'spotify:playlist:37i9dQZF1DXbTxeAdrVG2l',
    '2000s': 'spotify:playlist:37i9dQZF1DX4o1oenSJRJd',
    '2010s': 'spotify:playlist:37i9dQZF1DX5Ejj0EkURtP',
    '2020s': 'spotify:playlist:37i9dQZF1DX2M1RktxUUHG'
}

## Retrieve Data

In [ ]:
# Initialize an empty list to store information about all tracks
all_tracks = []

# Function to fetch all tracks from a playlist
def fetch_all_tracks(playlist_uri):
    # Fetch the initial set of tracks from the playlist
    results = sp.playlist_tracks(playlist_uri)
    tracks = results['items']
    print(f"Fetched {len(tracks)} tracks initially from {playlist_uri}")

    # Handle pagination to fetch all tracks if there are multiple pages
    while results['next']:
        time.sleep(1)  # Add a delay to avoid hitting the API rate limit
        results = sp.next(results)  # Fetch the next page of results
        tracks.extend(results['items'])  # Add new tracks to the list
        print(f"Fetched {len(tracks)} tracks so far from {playlist_uri}")

    return tracks

# Fetch tracks from each playlist in the 'playlists' dictionary
for decade, playlist_uri in playlists.items():
    print(f"Fetching tracks for the {decade} playlist")
    results = fetch_all_tracks(playlist_uri)

    # Extract and store track information from the fetched results
    for item in results:
        track = item['track']
        track_id = track['id']
        track_name = track['name']
        artist_name = track['artists'][0]['name']
        artist_id = track['artists'][0]['id']
        duration_ms = track['duration_ms']
        popularity = track['popularity']
        explicit = track['explicit']
        release_date = track['album']['release_date']
        release_year = release_date.split('-')[0]  # Extract year from release date

        # Fetch artist information
        artist_info = sp.artist(artist_id)
        genres = artist_info['genres'] if artist_info['genres'] else ['Unknown']

        # Create a dictionary with track information and append it to the list
        track_info = {
            'Decade': decade,
            'Track_ID': track_id,
            'Artist_name': artist_name,
            'Track_name': track_name,
            'Popularity': popularity,
            'Explicit': explicit,
            'Release_year': release_year,
            'Genres': genres
        }
        all_tracks.append(track_info)
    print(f"Completed fetching {len(results)} tracks for the {decade} playlist")

# Function to fetch audio features for a list of track IDs
def fetch_audio_features(track_ids):
    features = sp.audio_features(tracks=track_ids)
    return features

# Fetch audio features in batches (Spotify API limitation on the number of requests)
batch_size = 100
all_features = []
for i in range(0, len(all_tracks), batch_size):
    print(f"Fetching audio features for batch {i // batch_size + 1}")
    batch = [track['Track_ID'] for track in all_tracks[i:i + batch_size]]  # Create a list of track IDs for the current batch
    audio_features = fetch_audio_features(batch)  # Fetch audio features for the batch
    all_features.extend(audio_features)  # Append the fetched features to the list
    time.sleep(1)  # Add a delay to avoid hitting the API rate limit

# Combine track information with corresponding audio features
for track, features in zip(all_tracks, all_features):
    if features is not None:
        track.update(features)  # Update the track dictionary with audio features

# Print the total number of tracks fetched
print(f"Total tracks fetched: {len(all_tracks)}")

# Print the first few tracks to verify the fetched data
for track in all_tracks[:10]:
    print(track)

# Calculate and print the number of tracks found per decade
tracks_per_decade = {}
for track in all_tracks:
    decade = track['Decade']
    if decade in tracks_per_decade:
        tracks_per_decade[decade] += 1
    else:
        tracks_per_decade[decade] = 1

for decade, count in tracks_per_decade.items():
    print(f"Total tracks for {decade}: {count}")

## Save as JSON and CSV

In [ ]:
# Save the list of track information to a JSON file
with open('tracks.json', 'w') as f:
    # Use json.dump() to write the 'all_tracks' list to the file in JSON format
    # 'w' mode ensures that the file is opened for writing, and will be created if it doesn't exist
    json.dump(all_tracks, f)